# Insert a set of contracts (clients Informations) for the authentication test

In [4]:
import datetime
from typing import Any, Dict
from src.models.ContractsModel import ContractsModel
from src.models.db_schemes import Contracts
from src.helpers.config import get_settings
from src.agents.CaseOrchestratorAgent.utils.build_container import get_container
from src.utils.pii_safe import hash_field

In [6]:
async def seed_contracts():
    container = await get_container()
    settings = get_settings()
    salt_hash = settings.PII_HASH_SALT

    contracts_model = await ContractsModel.create_instance(db_client=container.db_client)

    seed_data: list[Dict[str, Any]] = [
        {
            "contract_number": hash_field("C-001", salt=salt_hash),
            "full_name": "Jon test",
            "birthday": datetime.date(1995, 5, 20),
            "postal_code": hash_field("22201", salt=salt_hash),
            "meta": {"mock": True, "source": "notebook"},
        },
        {
            "contract_number": hash_field("C-002", salt=salt_hash),
            "full_name": "Bob Example",
            "birthday": datetime.date(1988, 11, 2),
            "postal_code": hash_field("22303", salt=salt_hash),
            "meta": {"mock": True, "source": "notebook"},
        },
        {
            "contract_number": hash_field("C-003", salt=salt_hash),
            "full_name": "Charlie Example",
            "birthday": datetime.date(1992, 3, 14),
            "postal_code": hash_field("60311", salt=salt_hash),
            "meta": {"mock": True, "source": "notebook", "city": "Frankfurt"},
        },
        {
            "contract_number": hash_field("C-004", salt=salt_hash),
            "full_name": "Dina Example",
            "birthday": datetime.date(2000, 9, 7),
            "postal_code": hash_field("80331", salt=salt_hash),
            "meta": {"mock": True, "source": "notebook", "city": "Munich"},
        },
    ]

    created = []
    for row in seed_data:
        # If your Contracts table has more required columns, add them here.
        contract = Contracts(**row)
        created_contract = await contracts_model.create_contract(contract)
        created.append(created_contract)

    print(f"Inserted {len(created)} contracts ✅")
    return created

In [7]:
await seed_contracts()

Inserted 4 contracts ✅


In [8]:
# Build the agent

from src.agents.CaseOrchestratorAgent.graph_builder import build_graph
from src.agents.CaseOrchestratorAgent.AgentState import AgentState

graph = build_graph()

In [10]:
# set LANGSMITH settings (Optional)
import os

s = get_settings()
if s.LANGCHAIN_API_KEY:
    os.environ["LANGSMITH_TRACING"] = "true" if s.LANGCHAIN_TRACING_V2 else "false"
    os.environ["LANGSMITH_API_KEY"] = s.LANGCHAIN_API_KEY
    os.environ["LANGSMITH_PROJECT"] = s.LANGCHAIN_PROJECT or "default"
    if s.LANGCHAIN_ENDPOINT:
        os.environ["LANGSMITH_ENDPOINT"] = s.LANGCHAIN_ENDPOINT

In [ ]:
import asyncio
from fastmcp import Client

# test the agent
email = {
    "from_email": "younis.eng.software@gmail.com",
    "to_email": "test@younus-alsaadi.de",
    "subject": "Meter reading + dynamic tariff question",
    "direction": "inbound",
    "body": """Hello,
     I'd like to submit my latest electricity meter reading and also ask about your dynamic tariff.

    - Meter number: LB-9876543
    - Reading date: 25.09.2025
    - Reading value: 2438 kWh


    Contract number = C-001
    My postal number, is 22201

    Questions:
    1) How does your dynamic tariff work?
    2) Would it make sense for a 2-person household with an induction stove and no EV?
    3) Do prices change hourly?
    4) Can I switch any time?

    I can't find my contract number right now — if you need it, please tell me what I should provide.

    Thanks a lot,
    Jon Candy
    """,
}

email_no_auth = {
    "from_email": "younis.eng.software@gmail.com",
    "to_email": "test@younus-alsaadi.de",
    "subject": "Meter reading + dynamic tariff question",
    "direction": "inbound",
    "body": """
        Can you explain the dynamic tariff?
        """,
}


email_reply = {
    "from_email": "younis.eng.software@gmail.com",
    "to_email": "test@younus-alsaadi.de",
    "subject": "Meter reading + dynamic tariff question",
    "direction": "inbound",
    "body": """,
    Hello,


    Contract number = C-001
    My postal number, is 22201


    Best regards
    Younus AL-Saadi

    On Tue, 30 Dec 2025 at 18:11, <test@younus-alsaadi.de> wrote:
    Postal code,

    I am the AI customer support assistant handling your case. To proceed with your request, we kindly ask you to provide the following information to verify your identity:
    - Contract number

    Please reply to this email with the requested details and ensure the case ID a96b72b9-f2ff-4660-a00e-7686eb843cc3 remains in the subject line.

    Thank you for your cooperation.

    Best regards,
    Customer Support Team
    """,
}

initial_state: AgentState = {
    "Message": email_no_auth
    ,
    "errors": [],
}

# Run once, return final state
final_state = await graph.ainvoke(initial_state)

print("\n=== FINAL STATE ===")
print(final_state)

# Helpful: show errors only
if final_state.get("errors"):
    print("\n=== ERRORS ===")
    for e in final_state["errors"]:
        print(e)

In [9]:
# MCP server email check every 10S, If new email, the agent starting work
MCP_URL = "http://127.0.0.1:8000/mcp"
async with Client(MCP_URL) as client:
    while True:
        r = await client.call_tool("_email_imap_search", {"mailbox": "INBOX", "criteria": "UNSEEN", "limit": 5})
        uids = getattr(r, "data", r) or []

        for uid in uids:
            out = await graph.ainvoke({"uid": uid, "mailbox": "INBOX"})
            print("\n--- NEW EMAIL ---")
            print("From:", out.get("email", {}).get("from"))
            print("Subject:", out.get("email", {}).get("subject"))
            print("Explanation:\n", out.get("explanation"))

        await asyncio.sleep(10)